In [1]:
# Packages 
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow_examples.models.pix2pix import pix2pix

from IPython.display import clear_output

import time

In [2]:
# get user name depending on computer
username = os.getlogin()

# path
image_test_path = os.path.join (r'P:\Projects\SPR998.41 - ES&S\Jetson TX2 Investigation\Working\Dataset\test\image')
mask_test_path = os.path.join (r'P:\Projects\SPR998.41 - ES&S\Jetson TX2 Investigation\Working\Dataset\test\mask')
image_train_path = os.path.join (r'P:\Projects\SPR998.41 - ES&S\Jetson TX2 Investigation\Working\Dataset\train\image')
mask_train_path = os.path.join (r'P:\Projects\SPR998.41 - ES&S\Jetson TX2 Investigation\Working\Dataset\train\mask')

In [3]:
# load images 
def load_image(file_path, size):
    image_array = []
    
    for filename in os.listdir(file_path):
               
        # get mask path and read mask
        image_path = os.path.join(file_path, filename)
        image_raw = cv2.imread(image_path)/255
          
        # Append to list
        image_array.append(tf.image.resize(image_raw, size))
        
    return np.stack(image_array, axis=0)

In [4]:
# load masks
def load_mask(file_path, size):
    mask_array = []
    
    for filename in os.listdir(file_path):
        
        # get mask path and read mask
        image_path = os.path.join(file_path, filename)
        image_raw = cv2.imread(image_path)
        image_raw = np.array(tf.image.resize(image_raw, size, method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)).astype('int32')
        
        # filter mask into single channel categories
        image_out = np.zeros([image_raw.shape[0],image_raw.shape[1],1])
        
        image_out[np.where(image_raw[:,:,1]==255)] = 1   # Grass Pixel [0,255,0]
        image_out[np.where(image_raw[:,:,0]==51)] = 2    # Vegetation/tree [51,102,102]
        image_out[np.where(image_raw[:,:,0]==170)] = 3   # Roads [170,170,170]
        image_out[np.where(image_raw[:,:,0]==255)] = 4   # sky [255,120,0]     
    
        # Append to list
        mask_array.append(image_out)
        
    return  np.stack(mask_array, axis=0)

In [5]:
size = [128, 128]
image_test = load_image(image_test_path, size)
mask_test = load_mask(mask_test_path, size)
image_train = load_image(image_train_path, size)
mask_train = load_mask(mask_train_path, size)

In [6]:
# create full data set
image_array = np.concatenate((image_train, image_test), axis=0)
mask_array = np.concatenate((mask_train, mask_test), axis=0)

In [7]:
num_image = len(image_array)
num_mask = len(mask_array)

if num_image != num_mask:
    raise ImportError('Image data and mask data do not match!')
else:
     train_size = num_image   
    
print(num_image)
print(num_mask)

366
366


In [8]:
# Some parameters
batch_size = 32
buffer_size = 1000

In [9]:
# Data augmentation techniques
# Flip the image randomly
@tf.function
def image_flip(input_image, input_mask):
    
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)
        
#     if tf.random.uniform(()) > 0.5:
#         input_image = tf.image.rot90(input_image, k=1)
#         input_mask = tf.image.rot90(input_mask, k=1)
        
#     if tf.random.uniform(()) > 0.5:
#         input_image = tf.image.flip_up_down(input_image)
#         input_mask = tf.image.flip_up_down(input_mask)
    
    return input_image, input_mask


# Change the the colour of image randomly
@tf.function
def image_color(input_image, input_mask):

    input_image = tf.image.random_hue(input_image, 0.08)
    input_image = tf.image.random_saturation(input_image, 0.6, 1.6)
    input_image = tf.image.random_brightness(input_image, 0.05)
    input_image = tf.image.random_contrast(input_image, 0.7, 1.3)
    
    return input_image, input_mask

In [10]:
# Clip image and make sure image and mask are in the correct dtype
@tf.function
def clip_images(input_image,input_mask):
    return (tf.cast(tf.clip_by_value(input_image, 0, 1), tf.float32), tf.cast(input_mask,tf.int32))

In [11]:
def dataset_transform(dataset, augmentations):
        
    # Add the augmentations to the dataset
    for f in augmentations:
        # Apply the augmentation, run 2 jobs in parallel.
        dataset = dataset.map(f, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
   
    dataset = dataset.map(clip_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    return dataset

In [12]:
# Define model downsampling 
OUTPUT_CHANNELS = 5

base_model = tf.keras.applications.MobileNetV2(input_shape=size + [3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [13]:
# Define model downsampling 
OUTPUT_CHANNELS = 5

base_model = tf.keras.applications.MobileNetV2(input_shape=size + [3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [14]:
# define upsampling using pix2pix.upsample(#filters, filter_size)
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

def unet_model(output_channels):

    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='valid', activation='softmax')  #64x64 -> 128x128
    
    last_last = tf.keras.layers.Cropping2D(cropping=((1, 0), (1, 0)), data_format="channels_last")

    inputs = tf.keras.layers.Input(shape=size + [3])
    x = inputs

    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    x = last(x)
    x = last_last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [15]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 64, 64, 96), 1841984     input_3[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8, 8, 512)    1476608     model_1[1][4]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 8, 8, 1088)   0           sequential[0][0]                 
                                                                 model_1[1][3]              

In [17]:
# the model_dir states where the graph and checkpoint files will be saved to
estimator_model = tf.keras.estimator.model_to_estimator(keras_model = model, \
                                                        model_dir = './Forest_CKPT')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Using config: {'_model_dir': './Forest_CKPT', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002113B4B3D30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master':

In [21]:
# Create another input function for from slices
def input_function(features,labels=None,shuffle=False, batch_size = 128):
    
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    
    # Map the train dataset with tranformation for data augmentation
    if shuffle:
        dataset = dataset_transform(dataset, [image_flip,image_color])
        
    dataset = dataset.map(lambda features, labels: ({'input_3':features}, labels))
    
    if shuffle:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

input_function(image_array, mask_array, True)

<BatchDataset shapes: ({input_3: (None, 128, 128, 3)}, (None, 128, 128, 1)), types: ({input_3: tf.float32}, tf.int32)>

In [22]:
# TRAINING 
EPOCHS = 15
STEPS = train_size // batch_size * EPOCHS

estimator_model.train(input_fn = lambda: input_function(image_array, mask_array, True), steps = STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='./Forest_CKPT\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ./Forest_CKPT\keras\keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 165 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Forest_CKPT\model.ckpt.
INFO:tensorflow:loss = 1.9963233, step = 0
INFO:tensorflow:Saving checkpoints for 46 into ./Forest_CKPT\model.ckpt.
INFO:tensorflow:Saving checkpoints for 92 into ./Forest_CKPT\model.ckpt.
INFO:tensorflow:global_step/sec: 0.0761725
INFO:tensorflow:loss = 0.21212316, step = 100 (1312.8